In [23]:
import face_recognition
import cv2
import os
import glob
import numpy as np
import time
import pickle
from Facerec import Facerec
import time

In [24]:
validation_path="D:\\si\\Faces for Training\\v1"

In [25]:
fr=Facerec()
file_path=os.path.join(validation_path,"ans.txt")

In [30]:
images_path = glob.glob(os.path.join("D:\\si\\Faces for Training\\Data for Testing", "*.*"))
flag=0
file=open(file_path,"w")
for img_path in images_path:
    img=cv2.imread(img_path)
    t=time.time()
    boxes,names,accs=fr.detect_known_faces(img)
    if len(boxes)==0:
        vt=time.time()-t
        save_path=os.path.join(validation_path,"UnableToDetect")
        bname=os.path.basename(img_path)
#         basename = os.path.basename(img_path)
        (fname, ext) = os.path.splitext(bname)
        file.write("Source Image : "+fname+"  ;  Number of faces : "+str(len(boxes))+"  ;  Recognized ID : Unable To Detect  ;  Confidence : 0.0  ; Time : "+str(vt)+'\n')
        
        if not(os.path.isdir(save_path)):
            os.mkdir(save_path)
        save=os.path.join(save_path,bname)
        cv2.imwrite(save,img)
    else:  
        m=0
        for i in range(len(names)):
            
            if names[i]!="Unknown":
                l=boxes[i][3]-boxes[i][1]
                b=boxes[i][0]-boxes[i][2]
                a=l*b
                if(m<a):
                    box=boxes[i]
                    name=names[i]
                    m=a
                    acc=accs[i]
                    flag=1
        if flag==0:
            box=boxes[0]
            name=names[0]
            acc=accs[0]
            
        bname=os.path.basename(img_path)
#         basename = os.path.basename(img_path)
        (fname, ext) = os.path.splitext(bname)
        img=cv2.rectangle(img,(box[3],box[0]),(box[1],box[2]),(0,255,0),2)
        img=cv2.putText(img,"{0:.2f}".format(acc*100),(box[3],box[0]-10), cv2.FONT_HERSHEY_DUPLEX, 1,(0,255,0),2)
        vt=time.time()-t
        save_path=os.path.join(validation_path,name)
        if not(os.path.isdir(save_path)):
            os.mkdir(save_path)
        save=os.path.join(save_path,name+"_"+"{0:.2f}".format(acc*100)+"_"+fname+".jpg")
        cv2.imwrite(save,img)
#         print("Source Image : "+fname+"  ;  Number of faces : "+str(len(boxes))+"  ;  Recognized ID : "+name+"  ;  Confidence : {0:.2f}".format(acc*100)+"  ; Time : "+str(vt)+'\n')
        file.write("Source Image : "+fname+"  ;  Number of faces : "+str(len(boxes))+"  ;  Recognized ID : "+name+"  ;  Confidence : {0:.2f}".format(acc*100)+"  ; Time : "+str(vt)+'\n\n')
        
        flag=0
    
file.close()